# Compound Classification Challenge
- input: smiles
- model: CNN

In [ ]:
#!pip install wandb

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import wandb

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

wandb.login()

wandb: Currently logged in as: rimiiii (use `wandb login --relogin` to force relogin)


cuda:0 is available


True

In [ ]:
# colab 연결
from google.colab import drive
drive.mount('/gdrive')

data_dir = '/gdrive/My Drive/data/compound/'

Mounted at /gdrive


# Data 로드

Let's load the compound data file.

In [ ]:
cmpd_df = pd.read_csv(data_dir+'cmpd.csv')
cmpd_df.head()

,inchikey,smiles,group,activity
0,FNHKPVJBJVTLMP-UHFFFAOYSA-N,CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...,train,active
1,CUDVHEFYRIWYQD-UHFFFAOYSA-N,CNC(=O)c1cccc2cc(Oc3ccnc4cc(OCC5(N)CC5)c(OC)cc...,train,active
2,TTZSNFLLYPYKIL-UHFFFAOYSA-N,Cc1cc2cc(Oc3ccnc(Nc4cccc(CS(=O)(=O)NCCN(C)C)c4...,test,active
3,UOVCGJXDGOGOCZ-UHFFFAOYSA-N,COc1cc2c(cc1F)C(c1ccccc1Cl)=Nc1c(n[nH]c1C)N2,train,active
4,CUIHSIWYWATEQL-UHFFFAOYSA-N,Cc1ccc(Nc2nccc(N(C)c3ccc4c(C)n(C)nc4c3)n2)cc1S...,test,active


# dataset, dataloader 준비
- input: character level sequence data (batch_size, 1, smiles_max_length, vocab_size)
- label: actvie, inaactive

In [ ]:
# character level의 vocabulary 만들기
vocab = set()
for mol in cmpd_df.smiles:
    vocab.update(list(mol))
vocab = sorted(vocab)
char_to_idx = {char: i for i, char in enumerate(vocab)}
print(f'Vocabulary size: {len(vocab)}')

Vocabulary size: 38


## 데이터셋 만들기

In [ ]:
class cnnDataset(Dataset):
    def __init__(self, df, vocab, partition, max_len=400): 
        self.smiles = df['smiles']
        self.activity = df['activity']
        self.vocab = vocab
        self.partition = partition

        char_to_idx = {char: i for i, char in enumerate(vocab)}

        # X 만들기
        self.X = np.zeros((len(self.smiles), max_len))
        for i, smiles in enumerate(self.smiles):
            for j, char in enumerate(smiles[:max_len]):
                self.X[i][j] = char_to_idx[char] + 1 # index 0: 0으로 padding된 값들

        if self.partition == 'train':
          # label 만들기
          self.label = self.activity.eq('active').astype(float).to_numpy()
    
    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, index):
        if self.partition == 'train':
          return self.X[index], self.label[index]
        elif self.partition == 'test':
          return self.X[index]

# 모델 만들기
- 모델구조
  - embedding layer
  - 2 convolution layers(+average pooling, batchnorm)
  - global max pooling
- [참고 논문](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2523-5)

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size=39):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(9, vocab_size))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(9, 64))
        self.avg_pool1 = nn.AvgPool2d(kernel_size=(2, 1))
        self.avg_pool2 = nn.AvgPool2d(kernel_size=(2, 1))
        self.global_max_pool = nn.MaxPool2d(kernel_size=(1, 64))
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.leakyrelu = nn.LeakyReLU()
        
    def forward(self, x):
        x = x.view(-1, 1, 400, 39)
        x = self.leakyrelu(self.conv1(x))
        x = self.leakyrelu(self.bn1(x))
        x = x.view(-1, 1, 392, 64) # batch_size, 1, characters, filters
        x = self.avg_pool1(x) # characters의 avg_pooling
        x = self.leakyrelu(self.conv2(x))
        x = self.leakyrelu(self.bn2(x))
        x = x.view(-1, 1, 188, 64) # batch_size, 1, characters, filters
        x = self.avg_pool2(x) # characters의 avg_pooling
        x = self.global_max_pool(x) #filters의 max_pooling
        x = x.flatten(start_dim=1)
        return x   
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        wandb.init()
        self.embedding = self.create_emb_layer()
        self.conv_forward = CNN()
        self.fc1 = nn.Linear(94, 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout2d(0.3)
        self.leakyrelu = nn.LeakyReLU()
        
    def create_emb_layer(self, vocab_size=39):
        emb_layer = nn.Embedding(vocab_size, vocab_size)
        weight_matrix = torch.zeros((vocab_size, vocab_size))
        for i in range(vocab_size):
            weight_matrix[i][i] = 1
        emb_layer.weight.requires_grad = True # embedding weight도 gradient 학습 ### 다시 확인할 것
        return emb_layer
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.conv_forward(x)
        x = self.leakyrelu(self.fc1(x))
        x = self.fc2(x)
        x = self.dropout(x)
        return x.squeeze(1)

In [ ]:
def reinitialize_weights(m):
  """
  모듈의 가중치를 xavier_normal로 초기화
  편차를 0으로 초기화
  """
  if isinstance(m, nn.Conv2d):
    nn.init.xavier_normal_(m.weight.data)
    m.bias.data.fill_(0)

  elif isinstance(m, nn.Linear):
    nn.init.xavier_normal_(m.weight.data)
    m.bias.data.fill_(0)

def calculate_acc(y_pred, y_test):
  """
  epoch별 accuracy 계산
  """
  y_pred_tag = torch.round(torch.sigmoid(y_pred))
  correct_results_sum = (y_pred_tag == y_test).float().sum()
  acc = correct_results_sum/y_test.shape[0]
  acc = torch.round(acc * 100)
  return acc

# Train

In [ ]:
def train(model, train_loader, criterion, optimizer, config):
  wandb.watch(model, criterion, log="all", log_freq=10)
  losses = []

  model.train()
  for epoch in tqdm(range(1, config.epochs+1)):
      epoch_loss = 0
      epoch_acc = 0
      for X_batch, y_batch in train_loader:
          X_batch, y_batch = X_batch.long(), y_batch.float()
          X_batch, y_batch = X_batch.to(device), y_batch.to(device)
          optimizer.zero_grad()
          y_pred = model(X_batch)
          loss = criterion(y_pred, y_batch)
          
          loss.backward()
          optimizer.step()
          
          # loss
          epoch_loss += loss.item()
          acc = calculate_acc(y_pred, y_batch)
          avg_loss = epoch_loss/len(train_loader)

          # acc
          epoch_acc += acc.item()
          avg_acc = epoch_acc/len(train_loader)
          
          wandb.log({'acc': avg_acc, 'loss': avg_loss}, step=epoch)
          
      if epoch % 10 == 0:
          losses.append(avg_loss)
          print(f'Epoch {epoch+0:03}: | Loss: {avg_loss:.5f} | Acc: {avg_acc:.3f}')

In [ ]:
def run(df, vocab, max_len, config):
  wandb.init(config=config, reinit=True)

  config = wandb.config

  # dataset loader
  train_loader = DataLoader(cnnDataset(df[df.group.eq('train')], 
                                     vocab = vocab, 
                                     partition = 'train', 
                                     max_len = max_len), 
                            batch_size = config.batch_size, 
                            shuffle = True)
  
  model = Net()
  model.apply(reinitialize_weights)
  model.to(device)
  print(model)

  criterion = nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=0.0001)

  train(model, train_loader, criterion, optimizer, config)
  return model

In [ ]:
# 하이퍼파라미터 config 세팅
config = {
    'epochs': 300,
    'batch_size': 128,
    'learning_rate': 0.005,
    'dataset': 'compound_df',
    'architecture': 'CNN',
    }

In [ ]:
model = run(cmpd_df, vocab, max_len=400, config=config)

Net(
  (embedding): Embedding(39, 39)
  (conv_forward): CNN(
    (conv1): Conv2d(1, 64, kernel_size=(9, 39), stride=(1, 1))
    (conv2): Conv2d(1, 64, kernel_size=(9, 64), stride=(1, 1))
    (avg_pool1): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
    (avg_pool2): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
    (global_max_pool): MaxPool2d(kernel_size=(1, 64), stride=(1, 64), padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (fc1): Linear(in_features=94, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout2d(p=0.3, inplace=False)
  (leakyrelu): LeakyReLU(negative_slope=0.01)
)


  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 010: | Loss: 0.37388 | Acc: 77.125
Epoch 020: | Loss: 0.28536 | Acc: 81.219
Epoch 030: | Loss: 0.26568 | Acc: 81.969
Epoch 040: | Loss: 0.26893 | Acc: 81.344
Epoch 050: | Loss: 0.23017 | Acc: 84.406
Epoch 060: | Loss: 0.22292 | Acc: 83.781
Epoch 070: | Loss: 0.24548 | Acc: 82.625
Epoch 080: | Loss: 0.25585 | Acc: 80.969
Epoch 090: | Loss: 0.21323 | Acc: 84.469
Epoch 100: | Loss: 0.23296 | Acc: 83.094
Epoch 110: | Loss: 0.22721 | Acc: 82.625
Epoch 120: | Loss: 0.21198 | Acc: 84.906
Epoch 130: | Loss: 0.21301 | Acc: 84.125
Epoch 140: | Loss: 0.26880 | Acc: 81.812
Epoch 150: | Loss: 0.25650 | Acc: 82.781
Epoch 160: | Loss: 0.21977 | Acc: 83.531
Epoch 170: | Loss: 0.23508 | Acc: 82.562
Epoch 180: | Loss: 0.21813 | Acc: 84.000
Epoch 190: | Loss: 0.20970 | Acc: 83.719
Epoch 200: | Loss: 0.30405 | Acc: 80.031
Epoch 210: | Loss: 0.21912 | Acc: 83.500
Epoch 220: | Loss: 0.25569 | Acc: 82.594
Epoch 230: | Loss: 0.25427 | Acc: 81.625
Epoch 240: | Loss: 0.20908 | Acc: 83.906
Epoch 250: | Los

In [ ]:
# 모델 저장
file_name = 'CNN_netv4.pth'
torch.save(model.state_dict(), data_dir+file_name)

In [ ]:
# 모델 로드
file_name = 'CNN_netv4.pth'
net = Net()
net.load_state_dict(torch.load(data_dir+file_name))
net.to(device)

acc,▁▃▆▇▆▆▆▇▇▇▇▇█▇▇▇█▇▇▆▇█▇██▇▆██▇▇▇▇▇▆▇▇▇▇▇
loss,█▆▃▂▄▂▃▂▂▂▁▂▁▂▂▂▁▁▂▃▂▁▂▁▁▁▃▁▁▂▂▁▁▂▂▁▃▁▁▂
acc,81.65625
loss,0.23127


Net(
  (embedding): Embedding(39, 39)
  (conv_forward): CNN(
    (conv1): Conv2d(1, 64, kernel_size=(9, 39), stride=(1, 1))
    (conv2): Conv2d(1, 64, kernel_size=(9, 64), stride=(1, 1))
    (avg_pool1): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
    (avg_pool2): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
    (global_max_pool): MaxPool2d(kernel_size=(1, 64), stride=(1, 64), padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (fc1): Linear(in_features=94, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout2d(p=0.3, inplace=False)
  (leakyrelu): LeakyReLU(negative_slope=0.01)
)

# Test

In [ ]:
test_loader = DataLoader(cnnDataset(cmpd_df[cmpd_df.group.eq('test')], 
                                      vocab = vocab, 
                                      partition = 'test',
                                      max_len = 400),
                          batch_size = 256, 
                          shuffle = False)
# test label 추출
y_test = cmpd_df[cmpd_df.group.eq('test')].activity.eq('active').astype(float).to_numpy()

In [ ]:
# test 데이터를 이용한 성능 평가
y_pred_list = []

model.eval()
with torch.no_grad():
    i = 0
    for X_batch in test_loader:
      i += 1
      X_batch = X_batch.long()
      X_batch = X_batch.to(device)
      y_pred = model(X_batch)
      y_pred = torch.sigmoid(y_pred)
      y_pred_list.append(y_pred.cpu().numpy())

y_pred_list = np.hstack(y_pred_list)

In [ ]:
#roc-auc score
roc_auc_score(y_test, y_pred_list)

0.7826534104212821